
##About Datasets and Pretained tensorflow model


1.   Dataset collected randomly from the internet (human posing for measurement) and managed to collect 85 images, which are annotated using a bounding box in Roboflow.
2.   Later performed Preprocessing and augmentation in the annotated images, since we've got a pretty small amount of dataset.
3. After applying several augmentations, our dataset was modified for further training and analysis.
4. Took the raw URL of the pre-trained model implementations and also downloaded the tensorflow object detection in CSV format for further prediction and to get circumferences, width, and length of the mentioned body part.




###Setup and Imports:

Start by setting up your Colab environment and importing necessary libraries:

**1. Installing the TensorFlow Object Detection API and 'tensorflow_io':**

        In the Colab notebook, in order to install the TensorFlow Object Detection API and 'tensorflow_io':

In [ ]:
!git clone https://github.com/tensorflow/models.git
!apt-get install -y protobuf-compiler
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.
!pip install tensorflow-io

fatal: destination path 'models' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


**2. Add TensorFlow Object Detection API to PYTHONPATH:**

        Adding the TensorFlow Object Detection API to the PYTHONPATH so that Python can find the necessary modules:


In [ ]:
#!pip install tensorflow==<desired_version>
!git clone https://github.com/tensorflow/models.git
sys.path.append('/content/models/official')

In [ ]:
import sys
sys.path.append('/content/models/research')
sys.path.append('/content/models/research/slim')

**3. Setup and Imports:**

Start by setting up your Colab environment and importing necessary libraries:

In [ ]:
# Import libraries
import os
import official
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
from official.some_module import some_function
from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops
from object_detection.utils.label_map_util import get_label_map_dict


**4. Load Your Model:**

    Loading the pre-trained TensorFlow Object Detection model. Providing the mpretrained modeol url to the model checkpoint that was pretrained in Roboflow.

In [ ]:
model_url = 'https://universe.roboflow.com/ds/8J5858NJP4?key=qMvFC4AHcw'
configs = config_util.get_configs_from_pipeline_file(os.path.join(model_url, 'pipeline.config'))
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(model_dir, 'checkpoint', 'ckpt-0')).expect_partial()


**5. Load the Dataset:**

    Loading TensorFlow Object Detection dataset where the dataset includes bounding box annotations and image data annotated and augmented from Roboflow.

In [ ]:
dataset_dir = 'path/to/your/dataset'
dataset = tf.data.TFRecordDataset(os.path.join(dataset_dir, 'test.record'))


**6. Define a Function to Perform Inference:**

    Creating a function to perform inference on the dataset and obtain bounding box predictions:

In [ ]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections


**7. Perform Inference and Measure Circumferences:**

    Loop through the pretrained dataset, perform inference, and calculating circumferences, width and length of specific body measuements of chest, waist, hip, shoulder, sleeve, shirt, neck, bicep, and wrist.

In [ ]:
# Define the approximate conversion factor (e.g., pixels to centimeters)
conversion_factor = 2.54

for raw_record in dataset:
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    image = tf.image.decode_jpeg(example.features.feature['image/encoded'].bytes_list.value[0], channels=3)
    image_np = image.numpy()

    input_tensor = tf.convert_to_tensor([np.asarray(image_np)])
    detections = detect_fn(input_tensor)

    # Access bounding box coordinates and classes
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)

    # Assuming class 1 corresponds to the person in your dataset
    person_boxes = boxes[classes == 1]

    for box in person_boxes:
        ymin, xmin, ymax, xmax = box
        height = (ymax - ymin) * conversion_factor
        width = (xmax - xmin) * conversion_factor

        # Calculating circumferences, width and length based on bounding box dimensions and giving Simplified assumption
        chest_circumference = height
        waist_circumference = height
        hip_circumference = height
        Shoulder_width =  width
        Sleeve_length = height
        Shirt_length = height
        neck_circumference = width
        Bicep_circumference = width
        wrist_circumference = width


        # Print or store the calculated circumferences
        print("Chest Circumference:", chest_circumference)
        print("Waist Circumference:", waist_circumference)
        print("Hip Circumference:", hip_circumference)
        print("Shoulder Width:", Shoulder_width)
        print("Sleeve length:", Sleeve_length)
        print("Shirt length:", Shirt_length)
        print("Neck Circumference:", neck_circumference)
        print("Neck Circumference:", Bicep_circumference)
        print("Wrist Circumference:", wrist_circumference)


